# Week 2 Data Wrangling

### Import Packages

In [1]:
#Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly as py
import datetime as dt
import scipy as scp
from scipy import stats
from scipy.stats import ttest_ind

import warnings
warnings.filterwarnings('ignore')

### Load Datasets

In [2]:
applications = pd.read_csv('../Data/application_record.csv')

In [3]:
records = pd.read_csv('../Data/credit_record.csv')

In [4]:
applications.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
applications.describe()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
count,4.385570e+05,438557.000000,4.385570e+05,438557.000000,438557.000000,438557.0,438557.000000,438557.000000,438557.000000,438557.000000
mean,6.022176e+06,0.427390,1.875243e+05,-15997.904649,60563.675328,1.0,0.206133,0.287771,0.108207,2.194465
std,5.716370e+05,0.724882,1.100869e+05,4185.030007,138767.799647,0.0,0.404527,0.452724,0.310642,0.897207
min,5.008804e+06,0.000000,2.610000e+04,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,1.000000
25%,5.609375e+06,0.000000,1.215000e+05,-19483.000000,-3103.000000,1.0,0.000000,0.000000,0.000000,2.000000
50%,6.047745e+06,0.000000,1.607805e+05,-15630.000000,-1467.000000,1.0,0.000000,0.000000,0.000000,2.000000
75%,6.456971e+06,1.000000,2.250000e+05,-12514.000000,-371.000000,1.0,0.000000,1.000000,0.000000,3.000000
max,7.999952e+06,19.000000,6.750000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000


In [6]:
records.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [7]:
applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

In [8]:
records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [9]:
# number of unique IDs in applications
len(set(applications['ID'])) 



438510

In [10]:
# number of unique IDs in credit records
len(set(records['ID'])) 


45985

In [11]:
# how many IDs the two tables share
len(set(applications['ID']).intersection(set(records['ID']))) 

36457

#### There are more data points in records frame than applications frame. It appears there are 36457 records that can be used for judgement.  merge the 2 on ID.

In [12]:
# subset applications data - drop variables that most likely won't affect the DV.
applications2 = applications.drop(['FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL'], axis=1)

In [13]:
# rename columns 
applications2.rename(columns={'CODE_GENDER': 'gender', 'FLAG_OWN_CAR' : 'ownsCar', 'FLAG_OWN_REALTY' : 'ownsRealty', 'CNT_CHILDREN' : 'numChildren', 'AMT_INCOME_TOTAL' : 'totalIncome', 'NAME_INCOME_TYPE' : 'incomeType', 'NAME_EDUCATION_TYPE' : 'eduLvl', 'NAME_FAMILY_STATUS': 'famStatus', 'NAME_HOUSING_TYPE' : 'housingType', 'DAYS_BIRTH' : 'daysBirth', 'DAYS_EMPLOYED' : 'daysEmpl', 'OCCUPATION_TYPE' : 'occupation', 'CNT_FAM_MEMBERS' : 'famSize'}, inplace = True)

In [14]:
# check for missing values
applications2.isna().sum()

ID                  0
gender              0
ownsCar             0
ownsRealty          0
numChildren         0
totalIncome         0
incomeType          0
eduLvl              0
famStatus           0
housingType         0
daysBirth           0
daysEmpl            0
occupation     134203
famSize             0
dtype: int64

In [15]:
# replace missing values for occupation type with "Not identified"
applications2['occupation'].fillna(value='Not identified', inplace=True)

In [16]:
# convert daysBirth to Age in years for clarity
applications2['ageYrs'] = np.ceil(pd.to_timedelta(applications2['daysBirth'], unit='D').dt.days / -365.25)
applications2.drop('daysBirth', axis = 1, inplace = True)

In [17]:
# convert daysEmpl to yrsEmpl to keep in the same units
applications2['yrsEmpl']=-applications2['daysEmpl']/365.2425
applications2.loc[applications2['yrsEmpl']<0,'yrsEmpl']=0
applications2.drop('daysEmpl', axis=1, inplace=True)

In [18]:
# Create unemployed indicator - according to source explanation, positive daysEmpl are unemployed. 1=has a job, 0=unemployed
applications2['UNEMPLOYED']=0
applications2.loc[-applications2['yrsEmpl']<0,'UNEMPLOYED']=1

In [19]:
applications2.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,occupation,famSize,ageYrs,yrsEmpl,UNEMPLOYED
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,Security staff,2.0,59.0,3.104787,1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,1.0,53.0,8.353354,1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,1.0,53.0,8.353354,1


In [20]:
applications2.describe()

,ID,numChildren,totalIncome,famSize,ageYrs,yrsEmpl,UNEMPLOYED
count,4.385570e+05,438557.000000,4.385570e+05,438557.000000,438557.000000,438557.000000,438557.000000
mean,6.022176e+06,0.427390,1.875243e+05,2.194465,44.294942,5.948110,0.828234
std,5.716370e+05,0.724882,1.100869e+05,0.897207,11.459522,6.563855,0.377177
min,5.008804e+06,0.000000,2.610000e+04,1.000000,21.000000,0.000000,0.000000
25%,5.609375e+06,0.000000,1.215000e+05,2.000000,35.000000,1.015763,1.000000
50%,6.047745e+06,0.000000,1.607805e+05,2.000000,43.000000,4.016510,1.000000
75%,6.456971e+06,1.000000,2.250000e+05,3.000000,54.000000,8.495725,1.000000
max,7.999952e+06,19.000000,6.750000e+06,20.000000,69.000000,47.998248,1.000000


In [21]:
applications2.UNEMPLOYED.unique()

array([1, 0], dtype=int64)

In [22]:
applications2.UNEMPLOYED.value_counts()

1    363228
0     75329
Name: UNEMPLOYED, dtype: int64

In [23]:
# merge the datasets
merged = pd.merge(applications2, records, on="ID")

In [24]:
merged.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,occupation,famSize,ageYrs,yrsEmpl,UNEMPLOYED,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,-1,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,-2,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,-3,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,-4,C


In [25]:
# check data types of all variables
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777715 entries, 0 to 777714
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              777715 non-null  int64  
 1   gender          777715 non-null  object 
 2   ownsCar         777715 non-null  object 
 3   ownsRealty      777715 non-null  object 
 4   numChildren     777715 non-null  int64  
 5   totalIncome     777715 non-null  float64
 6   incomeType      777715 non-null  object 
 7   eduLvl          777715 non-null  object 
 8   famStatus       777715 non-null  object 
 9   housingType     777715 non-null  object 
 10  occupation      777715 non-null  object 
 11  famSize         777715 non-null  float64
 12  ageYrs          777715 non-null  float64
 13  yrsEmpl         777715 non-null  float64
 14  UNEMPLOYED      777715 non-null  int64  
 15  MONTHS_BALANCE  777715 non-null  int64  
 16  STATUS          777715 non-null  object 
dtypes: float64

In [26]:
merged.describe()

,ID,numChildren,totalIncome,famSize,ageYrs,yrsEmpl,UNEMPLOYED,MONTHS_BALANCE
count,7.777150e+05,777715.000000,7.777150e+05,777715.000000,777715.000000,777715.000000,777715.000000,777715.000000
mean,5.078743e+06,0.428082,1.885348e+05,2.208837,44.640988,6.364099,0.835451,-19.373564
std,4.180442e+04,0.745755,1.016225e+05,0.907380,11.240983,6.679963,0.370773,14.082208
min,5.008804e+06,0.000000,2.700000e+04,1.000000,21.000000,0.000000,0.000000,-60.000000
25%,5.044568e+06,0.000000,1.215000e+05,2.000000,35.000000,1.180038,1.000000,-29.000000
50%,5.069530e+06,0.000000,1.620000e+05,2.000000,44.000000,4.605160,1.000000,-17.000000
75%,5.115551e+06,1.000000,2.250000e+05,3.000000,54.000000,9.013190,1.000000,-8.000000
max,5.150487e+06,19.000000,1.575000e+06,20.000000,69.000000,43.020733,1.000000,0.000000


In [27]:
# look at unique status types for applicant's records
merged['STATUS'].unique()

array(['C', '1', '0', 'X', '5', '4', '3', '2'], dtype=object)

#### From source explanation: C= account paid (up-to-date), X= 0 balance for the month, 0= payment overdue 1-29 days, 1= payment overdue 30-59 days, 2= payment overdue 60-89 days, 3= payment overdue 90-119 days, 4= overdue 120-149 days, 5= overdue >150 days. We will call anything more than 60 days payment past due delinquent/bad. 

In [28]:
merged.STATUS.value_counts()

C    329536
0    290654
X    145950
1      8747
5      1527
2       801
3       286
4       214
Name: STATUS, dtype: int64

In [29]:
# find unique months balance values
merged['MONTHS_BALANCE'].unique()

array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
       -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
       -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
       -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
       -52, -53, -54, -55, -56, -57, -58, -59, -60], dtype=int64)

#### The majority of the applicants are paid up to date, have a 0 balance, or are overdue on payment by 1-29 days.

## recode status types to account for 0 days late cutoff. 1 = 'high risk' for any month where the individual's payment is overdue, and 0 is 'low risk'.

In [30]:
merged['GoodBad'] = merged['STATUS']
merged = merged.replace({'GoodBad' :{'C' : 'Good','X' : 'Good','0' : 'Bad', '1' : 'Bad','2' : 'Bad','3' : 'Bad','4' : 'Bad','5' : 'Bad'}})

In [31]:
mergedRes = merged.value_counts(subset=['ID', 'GoodBad']).unstack(fill_value=0).reset_index()

In [32]:
mergedRes.head()

GoodBad,ID,Bad,Good
0,5008804,2,14
1,5008805,2,13
2,5008806,7,23
3,5008808,2,3
4,5008809,0,5


In [33]:
# Calculate approval/rejection status based on applicant history majority
mergedRes.loc[(mergedRes['Good'] / mergedRes['Bad'] >= 1), 'ApprStatus'] = 1
mergedRes.loc[(mergedRes['Good'] / mergedRes['Bad'] < 1 ), 'ApprStatus'] = 0
mergedRes['ApprStatus'] = mergedRes['ApprStatus'].astype(int)
# combine datasets
merged3 = merged.merge(mergedRes, how='inner', on=['ID'])

In [34]:
merged3.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,...,famSize,ageYrs,yrsEmpl,UNEMPLOYED,MONTHS_BALANCE,STATUS,GoodBad,Bad,Good,ApprStatus
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,33.0,12.435574,1,0,C,Good,2,14,1
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,33.0,12.435574,1,-1,C,Good,2,14,1
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,33.0,12.435574,1,-2,C,Good,2,14,1
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,33.0,12.435574,1,-3,C,Good,2,14,1
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,2.0,33.0,12.435574,1,-4,C,Good,2,14,1


In [35]:
# convert continuous variables to integers
merged3.numChildren = merged.numChildren.astype(int)
merged3.totalIncome = merged.totalIncome.astype(int)
merged3.ageYrs = merged.ageYrs.astype(int)
merged3.yrsEmpl = merged.yrsEmpl.astype(int)
merged3.famSize = merged.famSize.astype(int)

In [36]:
# save 
merged3.to_csv("../Data/mergedGoodBad.csv")

In [37]:
# calculate a rate of good debts for ea user to compare categorical variables to e/o
merged3['rate'] = (merged3['Good']+0.1) / (merged3['Bad']+0.1)

In [38]:
merged3.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,...,ageYrs,yrsEmpl,UNEMPLOYED,MONTHS_BALANCE,STATUS,GoodBad,Bad,Good,ApprStatus,rate
0,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,33,12,1,0,C,Good,2,14,1,6.714286
1,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,33,12,1,-1,C,Good,2,14,1,6.714286
2,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,33,12,1,-2,C,Good,2,14,1,6.714286
3,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,33,12,1,-3,C,Good,2,14,1,6.714286
4,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,33,12,1,-4,C,Good,2,14,1,6.714286


In [39]:
# create dataFrame with merged data: drop MONTHS_BALANCE, STATUS, GoodBad, Bad, Good cols first

merged4Analysis = merged3[['ID', 'gender', 'ownsCar', 'ownsRealty', 'numChildren', 'totalIncome', 'incomeType', 'eduLvl', 'famStatus', 'housingType', 'occupation', 'famSize', 'ageYrs', 'yrsEmpl', 'UNEMPLOYED', 'ApprStatus', 'rate']]

In [40]:
# save dataset that includes rate for data exploration
merged4Analysis.to_csv("../Data/Rate.csv")

In [38]:
# save dataset for STRICT CUTOFF
merged4Analysis.to_csv("../Data/merged4AnalysisSTRICT.csv")

In [39]:
# Extract how many months account has been open for
merged4=pd.DataFrame(merged.groupby(['ID'])['MONTHS_BALANCE'].agg(min)).reset_index()

# Rename column
merged4.rename(columns={'MONTHS_BALANCE':'ACCOUNT_LENGTH'}, inplace=True)

# Make values positive
merged4['ACCOUNT_LENGTH']=-merged4['ACCOUNT_LENGTH']



In [40]:
merged4.head()

,ID,ACCOUNT_LENGTH
0,5008804,15
1,5008805,14
2,5008806,29
3,5008808,4
4,5008809,26


In [41]:
merged3.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,...,famSize,ageYrs,yrsEmpl,UNEMPLOYED,MONTHS_BALANCE,STATUS,GoodBad,Bad,Good,ApprStatus
0,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,2,33,12,1,0,C,Good,2,14,1
1,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,2,33,12,1,-1,C,Good,2,14,1
2,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,2,33,12,1,-2,C,Good,2,14,1
3,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,2,33,12,1,-3,C,Good,2,14,1
4,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,...,2,33,12,1,-4,C,Good,2,14,1


In [44]:
# sort dataFrame by ID so that each unique applicants' info appears together
mergedID = merged3.sort_values('ID')

In [45]:
mergedID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777715 entries, 0 to 171517
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              777715 non-null  int64 
 1   gender          777715 non-null  object
 2   ownsCar         777715 non-null  object
 3   ownsRealty      777715 non-null  object
 4   numChildren     777715 non-null  int32 
 5   totalIncome     777715 non-null  int32 
 6   incomeType      777715 non-null  object
 7   eduLvl          777715 non-null  object
 8   famStatus       777715 non-null  object
 9   housingType     777715 non-null  object
 10  occupation      777715 non-null  object
 11  famSize         777715 non-null  int32 
 12  ageYrs          777715 non-null  int32 
 13  yrsEmpl         777715 non-null  int32 
 14  UNEMPLOYED      777715 non-null  int64 
 15  MONTHS_BALANCE  777715 non-null  int64 
 16  STATUS          777715 non-null  object
 17  GoodBad         777715 non-nu